# Bivariate EDA: Crash Severity vs Single Factors

This notebook uses the cleaned crash dataset to explore how **Crash Severity** varies with one factor at a time (e.g., area type, functional class, lighting, driver/vehicle factors). In this **Step 4**, we read the cleaned data created in the data‑quality notebook and do not perform any additional cleaning here.

# 4.1 Setup and load cleaned data

In [ ]:
# Setup and load cleaned data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

cleaned_df = pd.read_pickle("../data/cleaned/crash_2018_cleaned.pkl")

cleaned_df["Crash Severity"].value_counts().sort_index()

# 4.2. Crash Severity across Spatial and Time Factors

## 4.2.1 Weekend, time of day, and area type

In [ ]:
spatial_time_vars = ["Weekend", "Time of Day", "Area Type"]

for var in spatial_time_vars:
    print(f"\nCrash severity (%) by {var}\n")
    ct = pd.crosstab(cleaned_df["Crash Severity"], cleaned_df[var], normalize="columns") * 100
    display(ct.round(1))

    fig, ax = plt.subplots(figsize=(7, 4))
    ct.T.plot(kind="bar", stacked=True, ax=ax)
    ax.set_title(f"Crash severity by {var}")
    ax.set_xlabel(var)
    ax.set_ylabel("Crash share by severity (%)")
    ax.legend(title="Crash severity", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

### Observations

- Severe crashes make up a slightly larger share of weekend crashes (Serious+Fatal ≈ 5.1%) than weekday crashes (≈ 3.5%), while PDO is a bit more common on weekdays.
- At nighttime, the share of Serious+Fatal crashes (≈ 5.6%) is almost double that in daytime (≈ 3.3%), indicating much higher severity risk after dark.
- In rural areas, Serious+Fatal crashes (≈ 9.3%) are over three times as common proportionally as in urban areas (≈ 2.3%), while urban areas have a higher PDO share.

## 4.2.2 Countywise distribution
Lets have a look on the counties experiencing maximum number of crashes and their severity distribution.

In [ ]:
# Total crashes per county
county_totals = cleaned_df.groupby("County", observed=False)["Area Type"].count()

top10_counties = county_totals.sort_values(ascending=False).head(10).index

# Pivot table of crash counts by severity for those counties
pivot_all = cleaned_df.pivot_table(
    index="County",
    columns="Crash Severity",
    values="Area Type",
    aggfunc="count",
    fill_value=0,
    observed=False,
)

top10_pivot = pivot_all.loc[top10_counties]

# Add total column and sort by it (optional but handy)
top10_pivot["Total"] = top10_pivot.sum(axis=1)
top10_pivot = top10_pivot.sort_values("Total", ascending=False)
print("Crash counts in top 10 counties and their severity\n")
top10_pivot

We may also look at the safer roads with counties experiencing minimal crash counts

In [ ]:
safe10_counties = county_totals.sort_values(ascending=True).head(10).index

# Pivot table of crash counts by severity for those counties
pivot_all = cleaned_df.pivot_table(
    index="County",
    columns="Crash Severity",
    values="Area Type",
    aggfunc="count",
    fill_value=0,
    observed=False,
)

safe10_pivot = pivot_all.loc[safe10_counties]

# Add total column and sort by it (optional but handy)
safe10_pivot["Total"] = safe10_pivot.sum(axis=1)
safe10_pivot = safe10_pivot.sort_values("Total", ascending=True)
print("Crash counts in top 10 safe counties and their severity\n")
safe10_pivot

Lets look at the counties experiencing higher percentage of severe crashes

In [ ]:
# county x severity table in row-wise percentages
county_sev = (
    cleaned_df.pivot_table(
        index="County",
        columns="Crash Severity",
        values="Area Type",  # or any column
        aggfunc="count",
        fill_value=0,
        observed=False,
    )  # <‑ add this
)
# convert to % of that county's crashes
county_sev_pct = county_sev.div(county_sev.sum(axis=1), axis=0) * 100
county_sev_pct.head()

In [ ]:
print("Top 10 ciunties by % of crash severity")
severity_colors = {
    "Fatal": "red",
    "Serious": "orange",
    "Minor": "mediumspringgreen",  # mint-ish green
    "PDO": "green",
}


def plot_county_severity_pct(county_sev_pct, severity, ax=None, top_n=10):
    if ax is None:
        ax = plt.gca()

    s = county_sev_pct[severity].sort_values(ascending=False).head(top_n)
    s = s.iloc[::-1]  # largest at top

    s.plot(kind="barh", ax=ax, color=severity_colors.get(severity, "steelblue"))

    ax.set_title(f"Top {top_n} counties: {severity} (% of county crashes)")
    ax.set_xlabel("Percent of crashes")
    ax.set_ylabel("County")
    ax.tick_params(axis="y", labelsize=8)


fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=False)
severities = ["Fatal", "Serious", "Minor", "PDO"]

for ax, sev in zip(axes, severities):
    plot_county_severity_pct(county_sev_pct, sev, ax=ax, top_n=10)

plt.tight_layout()
plt.show()

### Observations

- Perry County’s roads can be considered among the safest, with only 98 crashes in the year and most of them (70) being PDO. In contrast, Jefferson County’s roads experienced the highest number of crashes (more than 34,000), with 84 fatal incidents, making it the most vulnerable.
- None of the top three counties by fatal-crash proportion (Greene, Clay, Lowndes) appear among the top ten counties by total crash count. This indicates that some areas have fewer crashes overall, but a larger share of those crashes are severe.
- Clay County is the 6th lowest county in the list when ranked by total crash count, with only 233 crashes, but 6 of them are fatal. This gives Clay County the second-highest percentage of fatal crashes relative to its total crashes.

# 4.3. Crash Severity across Roadway and Traffic Conditions

### 4.3.1 Categorical roadway factors

In [ ]:
road_cat_vars = ["Functional Class Recode", "Curvature", "Grade", "Raised Median"]

for var in road_cat_vars:
    print(f"\nCrash severity (%) by {var}\n")
    ct = pd.crosstab(cleaned_df["Crash Severity"], cleaned_df[var], normalize="columns") * 100
    display(ct.round(1))

    fig, ax = plt.subplots(figsize=(7, 4))
    ct.T.plot(kind="bar", stacked=True, ax=ax)
    ax.set_title(f"Crash severity by {var}")
    ax.set_xlabel(var)
    ax.set_ylabel("Crash share by severity (%)")
    ax.legend(title="Crash severity", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

### 4.3.2 Numerical roadway factors

In [ ]:
road_num_vars = [
    "AADT",
    "Number of Lanes Num",
    "Number of Vehicles Num",
    "Speed Limit Num",
    "Impact Speed Num",
]

summary_road_num = cleaned_df.groupby("Crash Severity", observed=False)[road_num_vars].agg(
    ["mean", "median", "std"]
)
summary_road_num

In [ ]:
# Boxplots by severity, 2 at a time for readability

num_pairs = [
    ("AADT", "Number of Lanes Num"),
    ("Number of Vehicles Num", "Speed Limit Num"),
    ("Impact Speed Num", None),
]

for v1, v2 in num_pairs:
    fig, axes = plt.subplots(1, 2 if v2 is not None else 1, figsize=(12, 4))
    if v2 is None:
        axes = [axes]

    cleaned_df.boxplot(column=v1, by="Crash Severity", ax=axes[0])
    axes[0].set_title(f"{v1} by crash severity")
    axes[0].set_xlabel("Crash severity")
    axes[0].set_ylabel(v1)

    if v2 is not None:
        cleaned_df.boxplot(column=v2, by="Crash Severity", ax=axes[1])
        axes[1].set_title(f"{v2} by crash severity")
        axes[1].set_xlabel("Crash severity")
        axes[1].set_ylabel(v2)

    plt.suptitle("")
    plt.tight_layout()
    plt.show()

### Observations
- Functional class: Severity percentages are fairly similar across classes, but collectors show the highest share of Serious+Fatal crashes (≈6.8%), while interstates have the lowest (≈3.4%).

- Curvature: Crashes on curved segments are much more severe; Serious+Fatal share jumps from about 3.3% (no curve) to 8.6% (curve).

- Grade: Presence of grade also increases severity; Serious+Fatal crashes rise from ≈3.2% (no grade) to ≈6.3% (grade).

- Raised median: Roads without a raised median have a somewhat higher Serious+Fatal share (≈4.7%) than roads with medians (≈3.5%).

- AADT: More severe crashes tend to occur on lower‑AADT roads; median AADT drops from about 17,700 (PDO) to 7,000–8,000 (Serious/Fatal).

- Number of lanes: Median lane count is similar (about 3) across all severities, suggesting little clear severity pattern by lanes.

- Number of vehicles: Median number of vehicles is 2 for PDO, Minor, and Serious, but only 1 for Fatal, hinting that single‑vehicle crashes may be more often fatal.

- Speed limit: Higher severity is associated with slightly higher speed limits; median limit increases from about 45 mph (PDO) to around 58 mph (Fatal).

- Impact speed: Median impact speed rises clearly with severity (≈23 mph PDO → ≈38 mph Minor → ≈48 mph Serious → ≈58 mph Fatal), indicating strong association between higher impact speed and more severe outcomes.

# 4.4. Crash Severity across Driver and Vehicle Factors

## 4.4.1 Categorical driver/vehicle variables

In [ ]:
driver_cat_vars = [
    "Vehicle Type Recode",
    "Driver Gender Recode",
    "Driver License Validity",
    "Crash Manner Recode",
    "BAC Available",
]

for var in driver_cat_vars:
    print(f"\nCrash severity (%) by {var}\n")
    ct = pd.crosstab(cleaned_df["Crash Severity"], cleaned_df[var], normalize="columns") * 100
    display(ct.round(1))

    fig, ax = plt.subplots(figsize=(8, 4))
    ct.T.plot(kind="bar", stacked=True, ax=ax)
    ax.set_title(f"Crash severity by {var}")
    ax.set_xlabel(var)
    ax.set_ylabel("Crash share by severity (%)")
    ax.legend(title="Crash severity", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

## 4.4.2 Numeric driver variables (main dataset)

In [ ]:
driver_num_main = ["Driver Age"]

summary_driver_main = cleaned_df.groupby("Crash Severity")[driver_num_main].agg(
    ["mean", "median", "std"]
)
summary_driver_main

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
cleaned_df.boxplot(column="Driver Age", by="Crash Severity", ax=ax)
ax.set_title("Driver age by crash severity")
ax.set_xlabel("Crash severity")
ax.set_ylabel("Driver age")
plt.suptitle("")
plt.tight_layout()
plt.show()

## 4.4.3 DUI subset: severity vs BAC

In [ ]:
# 1. DUI subset
dui_df = cleaned_df[cleaned_df["BAC Available"]].copy()

# 2. BAC bins (adjust cut points if needed)
bac_bins = [-0.001, 0.0001, 0.08, 0.15, 10]  # 0, up to 0.08, 0.08–0.15, >0.15
bac_labels = ["0.000", "< 0.08", "0.08–0.15", "> 0.15"]

dui_df["BAC Category"] = pd.cut(dui_df["Driver BAC"], bins=bac_bins, labels=bac_labels)

# 3. Crosstab on BAC Category instead of raw BAC
ct_bac = pd.crosstab(dui_df["Crash Severity"], dui_df["BAC Category"], normalize="columns") * 100

display(ct_bac.round(1))

fig, ax = plt.subplots(figsize=(6, 4))
ct_bac.T.plot(kind="bar", stacked=True, ax=ax)
ax.set_title("Crash severity by BAC category (DUI subset)")
ax.set_xlabel("BAC category")
ax.set_ylabel("Crash share by severity (%)")
ax.legend(title="Crash severity", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

### Observations
- Vehicle type: Motorcycle crashes are far more severe: only 25% are PDO, while about 49% are Minor/Serious and 5% are Fatal, much higher fatal share than other vehicle types.

- Driver gender: Males show a slightly higher share of Serious+Fatal crashes (≈4.5%) than females (≈3.5%), but differences are modest.

- License validity: Drivers without valid licenses have a higher Serious+Fatal share (≈5.2%) compared with licensed drivers (≈3.9%).

- Crash manner: Head‑on and “Others” crashes are much more severe (Serious+Fatal ≈ 8–9%) than rear‑end or sideswipe/angle crashes (≈2–3%).

- BAC available: When BAC is available (likely DUI‑related cases), the Fatal share jumps to 5.5% compared with 0.5% when BAC data is absent, indicating much higher severity in this subset.

- Driver age: Median driver age is in the mid‑30s for PDO, Minor, and Serious crashes, and slightly higher (≈38.5) for Fatal crashes; age differences across severity levels are relatively small.

- Within the DUI subset, crashes with non‑zero BAC levels especially very high BAC (>0.15) show a noticeably higher share of Serious and Fatal outcomes than the overall crash population, indicating that DUI crashes tend to be more severe.

# 4.5. Crash Severity across Environmental Conditions

In [ ]:
env_cat_vars = [
    "Visibility Obstruction Recode",
    "Lighting Conditions Recode",
]

for var in env_cat_vars:
    print(f"\nCrash severity (%) by {var}\n")
    ct = pd.crosstab(cleaned_df["Crash Severity"], cleaned_df[var], normalize="columns") * 100
    display(ct.round(1))

    fig, ax = plt.subplots(figsize=(8, 4))
    ct.T.plot(kind="bar", stacked=True, ax=ax)
    ax.set_title(f"Crash severity by {var}")
    ax.set_xlabel(var)
    ax.set_ylabel("Crash share by severity (%)")
    ax.legend(title="Crash severity", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

### Observations
- Visibility obstructions are associated with a slightly lower share of recorded Severe (Serious+Fatal) crashes than “No obstruction,” but the difference is small and may reflect reporting or exposure patterns rather than true safety.

- Crashes in dark conditions have a much higher Severe share (Serious+Fatal ≈ 9.2%) than in daylight (≈ 3.5%), suggesting substantially greater severity risk at night.

- Dusk/dawn and illuminated/night‑lighted conditions fall between dark and daylight, but still show higher Severe shares than daylight alone.